In [ ]:
import pandas as pd 
import numpy as np

### 절대모멘텀
- 파일 로드 
- 파생변수 'STD-YM' 생성 -> Date 컬럼의 값을 년-월 추출 대입
- 년,월 별 마지막날 month_last_dt 데이터프레임 삽입 
- 전월의 종가의 값을 가지는 파생변수 생성
- 전년도의 종가의 값을 가지는 파생변수 생성
- 전월, 전년도의 종가를 가지고 거래내역 생성
- 수익율 계산

In [ ]:
df = pd.read_csv("../csv/SPY.csv")
df.head(1)

In [ ]:
## 새로운 파생변수 STD-YM 생성
## Date 컬럼을 기준으로 년-월 추출해서 파생변수 대입

## Date 컬럼을 데이터형을 datetime 변경 ( pd.to_datetime() )
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
## STD-YM 컬럼을 만들어서 Date컬럼에서 년(4자리)-월 추출
df['STD-YM'] = df['Date'].dt.strftime("%Y-%m")

In [ ]:
month_last_df = pd.DataFrame()

month_list = df['STD-YM'].unique()

for i in month_list:
    last_df = df[df['STD-YM'] == i].tail(1)
    month_last_df = pd.concat([month_last_df, last_df])
month_last_df.info()

In [ ]:
## 년-월 별 마지막날의 데이터들을 추출 
## 새로운 데이터프레임에 결합
month_last_df = df[~(df.shift(-1)['STD-YM'] == df['STD-YM'])]
month_last_df.info()

In [ ]:
## 'Date' 컬럼을 인덱스 변환
month_last_df.set_index(['Date'], inplace=True)

In [ ]:
month_last_df.head(10)

In [ ]:
## month_last_df 파생변수 2개 생성
## 'BF_1M' 전월의 종가(Adj Close)의 값
## 'BF_12M' 전년도의 종가(Adj Close)의 값
## NaN이 나온다면 0으로 대체
month_last_df["BF_1M"] = month_last_df.shift(1)['Adj Close'].fillna(0)
month_last_df["BF_12M"] = month_last_df.shift(12)['Adj Close'].fillna(0)
month_last_df.iloc[10:15]

In [ ]:
month_last_df.loc['1993-01-29':'1993-09-30']

In [ ]:
df.loc[1]

### loc, iloc
1. loc 
    - index와 column의 값을 비교하여 True인 행과 열을 출력
    - loc[index, columns]
    - loc[1] -> index의 값이 1인 행 출력
    - loc[:, column명]
2. iloc
    - index와 column의 위치를 출력
    - 데이터프레임에 2번째 행 출력 -> iloc[1]
    - 데이터프레림에서 2번째 열 출력 -> iloc[:,1]

In [ ]:
df.set_index(['Date'], inplace=True)

In [ ]:
## 거래내역 추가
## 구매 조건 -> 전월 종가 / 전년도 종가 - 1의 값이 0보다 크고 무한대가 아닌 경우
df['trade'] = ""

for i in month_last_df.index:
    signal = ""

    # 절대 모멘텀 계산
    momentum_index = month_last_df.loc[i, "BF_1M"] /\
         month_last_df.loc[i, "BF_12M"] - 1
    # print(momentum_index)
    # 절대 모멘텀 지표에 따라서 True / False 구분
    flag = True if((momentum_index > 0) and (momentum_index != np.inf) 
    and (momentum_index != -np.inf)) else False

    if flag:
        signal = 'buy'
    print("날짜 :", i, '모멘텀 인덱스 :', momentum_index, 
    "flag :", flag, 'signal :', signal)
    df.loc[i, "trade"] = signal
df['trade'].value_counts()
    


In [ ]:
rtn = 1.0
df['return'] = 1
buy = 0
sell = 0
for i in df.index:
    ##구매한 날짜를 체크 : 현재 행의 trade = 'buy' 전 행의 trade = ""인 경우
    if df.loc[i, 'trade'] == 'buy' and df.shift(1).loc[i, 'trade'] == "":
        buy = df.loc[i, 'Adj Close']
        print("구매일 :", i, "구매 가격 :", buy)
    elif df.loc[i, 'trade'] == "" and df.shift(1).loc[i, 'trade'] == 'buy':
        sell = df.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1
        df.loc[i, 'return'] = rtn
        print("판매일 : ", i, "판매 가격 :", sell, "수익율 :", rtn)

    if df.loc[i, 'trade'] == "":
        buy = 0 
        sell = 0 


In [ ]:
acc_rtn = 1 

for i in df.index:
    rtn = df.loc[i, 'return']
    acc_rtn *= rtn
    df.loc[i, 'acc_rtn'] = acc_rtn

print('누적 수익율 :', acc_rtn)

### 절대 모멘텀 함수로 구현
1. 1번 함수
    - 매개변수 2개 ( 데이터프레임, 기준이 되는 컬럼 )
    - 데이터프레임에 결측치와 이상치 제거
    - Date, 기준이 되는 컬럼 두 컬럼만 두고 나머지 컬럼은 삭제
    - Date 컬럼의 데이터를 시계열데이터 변경
    - 'STD-YM' 파생변수를 생성을 하여 년-월 대입
    - 인덱스를 Date로 변경
    - 수정이 된 데이터프레임을 리턴
    - 1번 함수 테스트하실때는 csv 파일 새로 불러와서 함수호출
2. 2번 함수
    - 매개변수(df) 1개( 데이터프레임 )
    - 새로운 데이터프레임 생성 
    - df의 데이터에서 년-월별 마지막 데이터들을 새로운 데이터프레임 삽입
    - 새로운 데이터프레임에 파생변수 2개 생성
    - 'BF_1M' : 전 월의 종가, 결측치는 0으로 대체
    - 'BF_12M' : 전 년의 종가, 결측치는 0으로 대체
    - 새로운 데이터프레임을 리턴


In [ ]:
'Adj Close' in df1.columns

In [ ]:
## 1번 함수 생성
def first(data, col):
    ##조건식을 이용하여 데이터형태를 같게 csv / yfinance
    # if type(data.index[0]) != int:
    if 'Date' not in data.columns:
        # data = data.reset_index()
        data.reset_index(inplace=True)

    ## 결측치와 이상치를 제거
    result = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)]
    result = result.loc[:, ['Date', col]] ## result[['Date', col]]
    result['Date'] = pd.to_datetime(result['Date'])
    result['STD-YM'] = result['Date'].dt.strftime("%Y-%m")
    result.set_index('Date', inplace=True)
    return result

In [ ]:
test_df = pd.read_csv("../csv/AAPL.csv")

test_df = first(test_df, 'Close')

In [ ]:
## 2번 함수
def second(data):
    result = pd.DataFrame()
    col = data.columns[0]
    data = data[~(data.shift(-1)['STD-YM'] == data['STD-YM'])]
    result = pd.concat([result, data])
    result['BF_1M'] = result.shift(1)[col].fillna(0)
    result['BF_12M'] = result.shift(12)[col].fillna(0)
    return result
    

In [ ]:
test_df_2 = second(test_df)

3. 3번 함수 생성
    - 매개변수 2개 (데이터프레임(df1), 데이터프레임(df2))
    - df1에 trade 컬럼 생성 값은 ""
    - df1에 return 컬럼 생성 값은 1
    - df2의 값들을 이용하여 momentum index를 구하고 df1에 거래 내역 대입
    - df1의 거래내역을 이용하여 수익율 return 대입
    - return의 값들은 가지고 누적 수익율을 acc_rtn 컬럼에 대입
    - 누적 수익율을 print()
    - df1을 리턴

In [ ]:
## 3번 함수
def third(df1, df2):
    df1["trade"] = ""
    df1['return'] = 1
    for i in df2.index:
        signal = ""

        # 절대 모멘텀 계산
        momentum_index = df2.loc[i, "BF_1M"] /\
            df2.loc[i, "BF_12M"] - 1
        # print(momentum_index)
        # 절대 모멘텀 지표에 따라서 True / False 구분
        flag = True if((momentum_index > 0) and (momentum_index != np.inf) 
        and (momentum_index != -np.inf)) else False

        if flag:
            signal = 'buy'
        print("날짜 :", i, '모멘텀 인덱스 :', momentum_index, 
        "flag :", flag, 'signal :', signal)
        df1.loc[i, "trade"] = signal

    rtn = 1.0
    buy = 0
    sell = 0
    col = df1.columns[0]
    for i in df1.index:
        ##구매한 날짜를 체크 : 현재 행의 trade = 'buy' 전 행의 trade = ""인 경우
        if df1.loc[i, 'trade'] == 'buy' and df1.shift(1).loc[i, 'trade'] == "":
            buy = df1.loc[i, col]
            print("구매일 :", i, "구매 가격 :", buy)
        elif df1.loc[i, 'trade'] == "" and df1.shift(1).loc[i, 'trade'] == 'buy':
            sell = df1.loc[i, col]
            rtn = (sell - buy) / buy + 1
            df1.loc[i, 'return'] = rtn
            print("판매일 : ", i, "판매 가격 :", sell, "수익율 :", rtn)

        if df1.loc[i, 'trade'] == "":
            buy = 0 
            sell = 0 
    
    acc_rtn = 1 

    for i in df1.index:
        rtn = df1.loc[i, 'return']
        acc_rtn *= rtn
        df1.loc[i, 'acc_rtn'] = acc_rtn

    print('누적 수익율 :', acc_rtn)

    return df1
    
    
    

In [ ]:
third(test_df, test_df_2)

In [ ]:
import yfinance as yf

In [ ]:
yf_df = yf.download('003490.KS', start='2018-01-01')

In [ ]:
df1 = first(yf_df, 'Close')
df1.head()

In [ ]:
import momentum as mmt

In [ ]:
mmt1 = mmt.Momentum(yf_df, 'Adj Close')

In [ ]:
mmt1.testing()

In [ ]:
t_df = pd.read_csv("../csv/AMZN.csv")

In [ ]:
mmt2 = mmt.Momentum(t_df, 'Adj Close')

In [ ]:
mmt2.testing()